In [1]:
import folium
from folium import CircleMarker
import geopandas as gpd
import importlib
import models
importlib.reload(models)
from models import get_conn
import pandas as pd
import elkai
pd.set_option('display.expand_frame_repr', False) # display full data in terminal
engine, Session = get_conn()

In [28]:
with Session() as session:
    waypoints = gpd.read_postgis("SELECT * FROM waypoints WHERE notes  = '2023 2024 Van Trip'", con=engine, geom_col='geometry')

# Travelling Salseman

# Move 'Denver Colorado' to the first position in waypoints
denver = waypoints[waypoints.title == 'Denver Colorado']
not_denver = waypoints[waypoints.title != 'Denver Colorado']
places = pd.concat([denver, not_denver]).reset_index(drop=True)

waypoints_dict = {}
for _, row in places.iterrows():
    waypoints_dict[row.title] = (row.latitude, row.longitude)

runs = 1

solved_cities = elkai.Coordinates2D(waypoints_dict).solve_tsp(runs=runs)


In [ ]:
# draw cities
center = [waypoints.latitude.mean(), waypoints.longitude.mean()]
map = folium.Map(location=center, zoom_start=5)

for _, point in waypoints.iterrows():
    folium.CircleMarker(
        location = [point.latitude, point.longitude],
        radius=3,
        color='blue',
        weight=1,
        fill=False,
        fill_color='blue',
        fill_opacity=0.2,
        tooltip=point.title,
    ).add_to(map)
map

# Draw TSP route
ordered_indices = [waypoints[waypoints.title == title].index[0] for title in solved_cities]
ordered_waypoints = waypoints.loc[ordered_indices].reset_index(drop=True)

coords = [[w.latitude, w.longitude] for _, w in ordered_waypoints.iterrows()]

folium.PolyLine(
    locations=coords,
    color='red',
    weight=2,
    opacity=1
).add_to(map)
map